# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [2]:
import findspark
findspark.init('/home/stuthi/prep/spark-2.0.0-bin-hadoop2.7/')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logreg').getOrCreate()

In [5]:
df = spark.read.csv('customer_churn.csv',header=True, inferSchema=True)

In [6]:
df.describe().show()

+-------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|summary|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|              Churn|
+-------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|  count|              900|              900|               900|              900|               900|                900|
|   mean|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|0.16666666666666666|
| stddev|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| 0.3728852122772358|
|    min|             22.0|            100.0|                 0|              1.0|               3.0|                  0|
|    max|             65.0|         18026.01|                 1|             9.15|              14.0|                  1|
+-------+---------------

In [7]:
df.head(1)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)]

In [8]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
from pyspark.ml.feature import VectorAssembler

In [13]:
assm = VectorAssembler(inputCols=['Age', 'Total_Purchase',
                                 'Account_Manager',
                                 'Years',
                                 'Num_Sites',
                                 ],outputCol='features')

In [14]:
fdf = assm.transform(df)

In [15]:
fdf.head(2)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1, features=DenseVector([42.0, 11066.8, 0.0, 7.22, 8.0])),
 Row(Names='Kevin Mueller', Age=41.0, Total_Purchase=11916.22, Account_Manager=0, Years=6.5, Num_Sites=11.0, Onboard_date=datetime.datetime(2013, 8, 13, 0, 38, 46), Location='6157 Frank Gardens Suite 019 Carloshaven, RI 17756', Company='Wilson PLC', Churn=1, features=DenseVector([41.0, 11916.22, 0.0, 6.5, 11.0]))]

In [16]:
fdf = fdf.select(['features','Churn'])

In [17]:
fdf.show(2)

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
+--------------------+-----+
only showing top 2 rows



In [21]:
train,test = fdf.randomSplit([0.7,0.3])

In [22]:
from pyspark.ml.classification import LogisticRegression

In [25]:
lr = LogisticRegression(labelCol='Churn')

In [26]:
logreg = lr.fit(train)

In [27]:
summ = logreg.summary

In [31]:
summ.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                613|                613|
|   mean| 0.1729200652528548|0.13050570962479607|
| stddev|0.37848699713291156|0.33713407475776114|
|    min|                  0|                0.0|
|    max|                  1|                1.0|
+-------+-------------------+-------------------+



In [32]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [55]:
predlabels = logreg.evaluate(test)

In [36]:
predlabels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|    0|[4.41732066440252...|[0.98807734567196...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.42241951852438...|[0.99560297121664...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.63651906537504...|[0.83706072938121...|       0.0|
|[28.0,11204.23,0....|    0|[1.90682344591687...|[0.87066185775802...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.75900314818908...|[0.94041980453763...|       0.0|
|[29.0,11274.46,1....|    0|[4.61355949757167...|[0.99018091275940...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.30158111697811...|[0.96448301252375...|       0.0|
|[30.0,8403.78,1.0...|    0|[5.99228898483060...|[0.99750828440037...|       0.0|
|[30.0,13473.35,0....|    0|[2.73670803862121...|[0.93915826566834...|       0.0|
|[31.0,5387.75,0

In [38]:
evall = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='prediction')

In [39]:
evall.evaluate(predlabels.predictions)

1.0

In [46]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [42]:
train.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                613|
|   mean| 0.1729200652528548|
| stddev|0.37848699713291156|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [43]:
test.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                287|
|   mean|0.15331010452961671|
| stddev| 0.3609154802977014|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [45]:
latest_model = lr.fit(fdf)

In [47]:
newtest = assm.transform(new_customers)

In [48]:
newtest.head(2)

[Row(Names='Andrew Mccall', Age=37.0, Total_Purchase=9935.53, Account_Manager=1, Years=7.71, Num_Sites=8.0, Onboard_date=datetime.datetime(2011, 8, 29, 18, 37, 54), Location='38612 Johnny Stravenue Nataliebury, WI 15717-8316', Company='King Ltd', features=DenseVector([37.0, 9935.53, 1.0, 7.71, 8.0])),
 Row(Names='Michele Wright', Age=23.0, Total_Purchase=7526.94, Account_Manager=1, Years=9.28, Num_Sites=15.0, Onboard_date=datetime.datetime(2013, 7, 22, 18, 19, 54), Location='21083 Nicole Junction Suite 332, Youngport, ME 23686-4381', Company='Cannon-Benson', features=DenseVector([23.0, 7526.94, 1.0, 9.28, 15.0]))]

In [53]:
newpred = latest_model.transform(newtest)

In [54]:
newpred.select(['Company','prediction']).show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

